In [1]:
import numpy as np
import matplotlib.pyplot as plt
import bruges as bg
from PIL import Image
import pickle
import random

In [2]:
def contains_all_nums(strata, max_layers):
    for i in range(0, max_layers):
        if i not in strata:
            return False
    return True

In [3]:
def calc_split_points(strata):
    split_points = np.array([random.random() for _ in range(2)])
    split_points *= random.randint(8, len(strata) - 2)/np.amax(split_points)
    
    split_points = np.round(split_points)
    split_points = np.sort(split_points)
    return split_points

In [4]:
def make_random_wedges():
    depth = (random.randint(600,1200), random.randint(800, 2000), random.randint(400,1200))
    width = (random.randint(600,1200), random.randint(800, 2000), random.randint(400,1200))
    mode = random.choice(['linear', 'clinoform'])
    conformance = random.choice(['top', 'bottom', 'proportional'])
    max_layers = random.randint(4, 10)
    
    strata = [random.randint(0, max_layers) for i in range(5, 20)]
    while not contains_all_nums(strata, max_layers):
        strata = [random.randint(0, max_layers) for i in range(5, 20)]
    
    threshold = random.randint(2, max(len(strata)//4,2))
    split_points = calc_split_points(strata).astype('int8')
    while split_points[1] - split_points[0] < threshold:
            split_points = calc_split_ŒŒpoints(strata).astype('int8')
    upper_strata = tuple(strata[:split_points[0]])
    middle_strata = tuple(strata[split_points[0]:split_points[1]])
    lower_strata = tuple(strata[split_points[1]:])
    starting_thickness = random.uniform(0, 2)
    thickness=(starting_thickness, starting_thickness + random.uniform(0,2))
    w, top, base, ref = bg.models.wedge(depth=depth,
                                    width=width,
                                    strat=tuple([upper_strata, middle_strata, lower_strata]),
                                    mode=mode,
                                    thickness=thickness,
                                   )
    return w

In [8]:
num_samples = 100
for i in range(num_samples):
    done = False
    while not done:
        try:
            w = make_random_wedges()
            rocks = np.array([i+1 for i in (np.unique(w))])
            # Fancy indexing into the rocks with the model.
            impedance = rocks[w-1]
            # Make reflectivity.
            rc = (impedance[1:] - impedance[:-1]) / (impedance[1:] + impedance[:-1])
            rc -= np.amin(rc)
            rc *= 255/(np.amax(rc)-np.amin(rc))


            impedance -= np.amin(impedance)
            impedance = impedance.astype('float64')
            impedance *= 255/(np.amax(impedance) - np.amin(impedance))
            impedance = np.round(impedance).astype('int8')
            Image.fromarray(rc).convert("RGB").convert("L").save("images/reflectivity_{}.png".format(i+1))
            Image.fromarray(impedance).convert("RGB").convert("L").save("images/impedance_{}.png".format(i+1))

            with open('data/reflectivity_{}.npy'.format(i+1), 'wb') as gen_file:
                np.save(gen_file, rc)
            with open('data/impedance_{}.npy'.format(i+1), 'wb') as gen_file:
                np.save(gen_file, impedance)
            done=True
        except:
            pass
